# Neural Networks in Keras

every time we need to use a neural network, we won't need to code the activation function, gradient descent, etc. There are lots of packages for this, which we recommend you to check out, including the following:

Keras
TensorFlow
Caffe
Theano
Scikit-learn
And many others!
In this course, we will learn Keras. Keras makes coding deep neural networks simpler. To demonstrate just how easy it is, you're going to build a simple fully-connected network in a few dozen lines of code.

We’ll be connecting the concepts that you’ve learned in the previous lessons to the methods that Keras provides.

The general idea for this example is that you'll first load the data, then define the network, and then finally train the network.

## Sequential Model

* [Official tutorial document](https://keras.io/getting-started/sequential-model-guide/) and [Sequential model API](https://keras.io/models/sequential/)

```python
from keras.models import Sequential

#Create the Sequential model
model = Sequential()
```
    
The [keras.models.Sequential class](https://keras.io/models/sequential/) is a wrapper for the neural network model that treats the network as a sequence of layers. It implements the Keras model interface with common methods like `compile()`, `fit()`, and `evaluate()` that are used to train and run the model. We'll cover these functions soon, but first let's start looking at the layers of the model.

## Layers

The Keras Layer class provides a common interface for a variety of standard neural network layers. There are fully connected layers, max pool layers, activation layers, and more. You can add a layer to a model using the model's add() method. For example, a simple model with a single hidden layer might look like this:

```python
    import numpy as np
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation

    # X has shape (num_rows, num_cols), where the training data are stored
    # as row vectors
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)

    # y must have an output vector for each input vector
    y = np.array([[0], [0], [0], [1]], dtype=np.float32)

    # Create the Sequential model
    model = Sequential()

    # 1st Layer - Add an input layer of 32 nodes with the same input shape as
    # the training samples in X
    model.add(Dense(32, input_dim=X.shape[1]))

    # Add a softmax activation layer
    model.add(Activation('softmax'))

    # 2nd Layer - Add a fully connected output layer
    model.add(Dense(1))

    # Add a sigmoid activation layer
    model.add(Activation('sigmoid'))
```

> Keras requires the input shape to be specified in the first layer and it will automatically infer the shape of all other layers. This means you only have to explicitly set the input dimensions for the first layer.

The first (hidden) layer from above, 

```python
model.add(Dense(32, input_dim=X.shape[1]))
```

creates 32 nodes which each expect to receive 2-element vectors as inputs. 

> Each layer in Keras takes the outputs from the previous layer as inputs and pipes through to the next layer. This chain of passing output to the next layer continues until the last layer, which is the output of the model. 

We can see that the output has dimension 1.

The activation "layers" in Keras are equivalent to specifying an activation function in the Dense layers. e.g.,

```python
model.add(Dense(128)); 
model.add(Activation('softmax')) 
```

is computationally equivalent to 

```python
model.add(Dense(128, activation="softmax"))
```
But it is common to explicitly separate the activation layers because it allows direct access to the outputs of each layer before the activation is applied (which is useful in some model architectures).

## Compilation

Before training a model, you need to configure the learning process, which is done via the `compile` function that is compiling the Keras model calls the backend (tensorflow, theano, etc.). The 'compile' function signature is shown below and it detailed API is described [here](https://keras.io/models/sequential/)

```python
compile(self, optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
```

The most important three arguments are:

* `An optimizer`. This could be the string identifier of an existing optimizer (such as rmsprop or adagrad), or an instance of the Optimizer class. See: [optimizers](https://keras.io/optimizers/).
* `A loss function`. This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as categorical_crossentropy or mse), or it can be an objective function. See: [losses](https://keras.io/losses).
* `A list of metrics`. For any classification problem you will want to set this to metrics=`accuracy`. A metric could be the string identifier of an existing metric or a custom metric function.

For example, we may specify the `loss function` to be categorical_crossentropy which can be used when there are only two classes, and specify adam as the `optimizer` (which is a reasonable default when speed is a priority). And finally, we can specify what `metrics` we want to evaluate the model with. Here we'll use accuracy.

```python
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.01), metrics = ["accuracy"])
```

We can see the resulting model architecture with the following command:

```python
model.summary()
```

## Training

Keras models are trained on Numpy arrays of input data and labels. For training a model, you will typically use the `fit` function. The 'fit' function has following signature and its detailed API is described [here](https://keras.io/models/sequential/)

```python
fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
```

The most important arguments are:

* `x`: Numpy array of training data (if the model has a single input), or list of Numpy arrays (if the model has multiple inputs). 
* `y`: Numpy array of target (label) data (if the model has a single output), or list of Numpy arrays (if the model has multiple outputs). 
* `batch_size`: Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32.
* `epochs`: Integer. Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided. 
* `verbose`: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
* `callbacks`: List of `keras.callbacks.Callback` instances. List of callbacks to apply during training. See [callbacks](https://keras.io/callbacks).
* `validation_split`: Float between 0 and 1. Fraction of the training data to be used as    validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. The validation data is selected from the last samples in the x and y data provided, before shuffling.
* `validation_data`: tuple (x_val, y_val) or tuple  (x_val, y_val, val_sample_weights) on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data.  validation_data will override  validation_split.

For example, we may specify `batch_size` to be 32 and specify epochs to be 20. Finally, we can specify `validation_split` to be 0.2, which means that we will use 0.8 of the original training data to train the model and 0.2 of the rest to validate the model being trained:

```python
model.fit(x_train_e, y_train, batch_size=32, epochs=20, validation_split=0.2)
```

Finally, we can use the following command to evaluate the model:

```python
model.evaluate()
```

## Callbacks

As aforementioned, we can specifiy a list of callbacks (as the keyword argument callbacks) to the .fit() function of the established model for training. You can use callbacks to get a view on internal states and statistics of the model during training. The relevant methods of the callbacks will then be called at each stage of the training.

A callback is a set of functions to be applied at given stages of the training procedure. Keras provides many built-in callbacks, which you can view [here](https://keras.io/callbacks/)

I will be describing the usage of three built-in callbacks here:
* ModelCheckpoint
* EarlyStopping
* History

### ModelCheckpoint

`ModelCheckpoint` checks, at each stage of training, whether certain metrics are achieved or not, and if they are, it saves the trained model (including it parameters) to the specified file.

```python
keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
```

Important arguments for this function:

* `filepath`: string, path to save the model file.
* `monitor`: quantity to monitor, can be `vac_loss` or `vac_acc`.
* `verbose`: verbosity mode, 0 or 1.
* `save_best_only`: if save_best_only=True, the latest best model according to the quantity monitored will not be overwritten.
* `mode`: one of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity. For val_acc, this should be max, for val_loss this should be min, etc. In auto mode, the direction is automatically inferred from the name of the monitored quantity.

For example, if we want to monitor validation accuracy while training the model and saves the model with name of my_model.hdf5 to current directory when the monitored validation accuracy reachs max (of all validation accuracies that have been monitored so far) at each epoch, we can create a `ModelCheckpoint` with following configuration: 

```python
checkpoint = ModelCheckpoint("./my_model.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
```

Then, we can feed this ModelCheckpoint callback to .fit function:

```python
model.fit(x_train_e, y_train, batch_size=32, epochs=20, validation_split=0.2, callbacks=[checkpoint])
```

### EarlyStopping

Early stopping is a regularization approach. It stops training when a monitored quantity has stopped improving and it has fuction signature:

```python
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)
```

Important arguments for this function:

* `monitor`: quantity to be monitored.
* `min_delta`: minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.
* `patience`: number of epochs with no improvement after which training will be stopped.
* `verbose`: verbosity mode.
* `mode`: one of {auto, min, max}. In min mode, training will stop when the quantity monitored has stopped decreasing; in  max mode it will stop when the quantity monitored has stopped increasing; in auto mode, the direction is automatically inferred from the name of the monitored quantity.

For example, if we want to monitor validation accuracy while training the model and stops training when the validation accuracy has stopped increasing for 4 number of epochs, we can create a `EarlyStopping` with following configuration:

early_stop = EarlyStopping(monitor="val_acc", mode="max", patience=4)

Then, we can feed this ModelCheckpoint callback along with ModelCheckpoint to .fit function:

```python
model.fit(x_train_e, y_train, batch_size=32, epochs=20, validation_split=0.2, callbacks=[checkpoint, early_stop])
```

### History

History is one of the default callbacks that are automatically registered when training deep learning models. For detailed explanation on history, please refer to [here](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/)

> It records training metrics for each epoch. This includes the loss and the accuracy (for classification problems) as well as the loss and accuracy for the validation dataset, if one is set.

The history object is returned from calls to the fit() function used to train the model. Metrics are stored in a dictionary in the history member of the object returned.

For example, you can list the metrics collected in a history object using the following snippet of code after a model is trained:

```python
# list all data in history
print(history.history.keys())
```
This may give us:
```python
['acc', 'loss', 'val_acc', 'val_loss']
```

The plots can provide an indication of useful things about the training of the model, such as:

* It’s speed of convergence over epochs (slope).
* Whether the model may have already converged (plateau of the line).
* Whether the mode may be over-fitting the training data (inflection for validation line).
* And more.

We can plot the accuracy and loss as follow:

```python
history = model.fit(x_train_e, y_train, batch_size=32, epochs=20, validation_split=0.2)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
```

<img src='images/history_training_dataset.png' height="300" width="400"/>

<img src='images/history_validation_dataset.png' height="300" width="400"/>

### Custom Callback

You can create a custom callback by extending the base class `keras.callbacks.Callback`. A callback has access to its associated model through the class property `self.model`.

You can overwrite following base function when you want to perform some work at appropriate time:
* `on_epoch_begin`: called at the beginning of every epoch.
* `on_epoch_end`: called at the end of every epoch.
* `on_batch_begin`: called at the beginning of every batch.
* `on_batch_end`: called at the end of every batch.
* `on_train_begin`: called at the beginning of model training.
* `on_train_end`: called at the end of model training.

The arguments these functions have respectively:

* `on_epoch_begin` and `on_epoch_end` expect two positional arguments: epoch, logs
* `on_batch_begin` and `on_batch_end` expect two positional arguments: batch, logs
* `on_train_begin` and `on_train_end` expect one positional argument: logs

For example, you can create a ROC and Accuracy evaluation callback as follow. It evaluates the ROC and Accuracy on the validation data on the end of each epoch.

```python
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=()):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = roc_auc_score(self.y_val, y_pred)
        print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))
```

Then, as normal callbacks, you can feed this custom callback to .fit function:

```python
ra_val = RocAucEvaluation(validation_data=(X_val, y_val))
model.fit(x_train_e, y_train, batch_size=32, epochs=20, validation_split=0.2, callbacks=[checkpoint, early_stop, ra_val])
```

## Optimizers

There are many optimizers in Keras, that we encourage you to explore further, in this link, or in this excellent [blog post](http://ruder.io/optimizing-gradient-descent/index.html). These optimizers use a combination of the tricks above, plus a few others. Some of the most common are:

**SGD**

This is Stochastic Gradient Descent. It uses the following parameters:

* Learning rate.
* Momentum (This takes the weighted average of the previous steps, in order to get a bit of momentum and go over bumps, as a way to not get stuck in local minima).
* Nesterov Momentum (This slows down the gradient when it's close to the solution).

**Adam**

Adam (Adaptive Moment Estimation) uses a more complicated exponential decay that consists of not just considering the average (first moment), but also the variance (second moment) of the previous steps.

**RMSProp**

RMSProp (RMS stands for `Root Mean Squared Error`) decreases the learning rate by dividing it by an exponentially decaying average of squared gradients.

## Concrete Example

Let's start with the simplest example. In this quiz you will build a simple multi-layer feedforward neural network to solve the XOR problem.

1. Set the first layer to a Dense() layer with an output width of 8 nodes and the input_dim set to the size of the training samples (in this case 2).
2. Add a tanh activation function.
3. Set the output layer width to 1, since the output has only two classes. (We can use 0 for one class an 1 for the other)
4. Use a sigmoid activation function after the output layer.
5. Run the model for 50 epochs.

This should give you an accuracy of 50%. That's ok, but certainly not great. Out of 4 input points, we're correctly classifying only 2 of them. Let's try to change some parameters around to improve. For example, you can increase the number of epochs. You'll pass this quiz if you get 75% accuracy. Can you reach 100%?

To get started, review the Keras documentation about models and layers. The Keras example of a [Multi-Layer Perceptron network](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) is similar to what you need to do here. Use that as a guide, but keep in mind that there will be a number of differences.

In [2]:
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
# Using TensorFlow 1.0.0; use tf.python_io in later versions
# tf.python.control_flow_ops = tf

# Set random seed
np.random.seed(42)

# Our data
X = np.array([[0,0],[0,1],[1,0],[1,1]]).astype('float32')
y = np.array([[0],[1],[1],[0]]).astype('float32')

# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation

# Building the model
xor = Sequential()

# Add required layers
xor.add(Dense(8, input_dim=2))
xor.add(Activation('tanh'))
xor.add(Dense(1))
xor.add(Activation('sigmoid'))

# Specify loss as "binary_crossentropy", optimizer as "adam",
# and add the accuracy metric
xor.compile(loss="binary_crossentropy", optimizer="adam", metrics = ["accuracy"])

# Uncomment this line to print the model architecture
xor.summary()

# Fitting the model
history = xor.fit(X, y, nb_epoch=1000, verbose=0)

# Scoring the model
score = xor.evaluate(X, y)
print("\nAccuracy: ", score[-1])

# Checking the predictions
print("\nPredictions:")
print(xor.predict_proba(X))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 24        
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


4/4 [==============================] - 0s

Accuracy:  1.0

Predictions:
4/4 [==============================] - 0s
[[ 0.06273437]
 [ 0.86689943]
 [ 0.91195548]
 [ 0.11260458]]


## References

[Display Deep Learning Model Training History in Keras](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/)
